In [2]:
import medleydb as mdb
import pandas as pd
import numpy as np
import yaml
import os

def generate_stem_key(x):
    if x < 10:
        return "S0%d" % x
    else:
        return "S%d" % x
def extract_stem_instruments(t):
    return {generate_stem_key(k): v.instrument for k, v in t.stems.items()}

/home/carlnys/data/medleydb/medleydb/__init__.py:64: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/home/carlnys/data/medleydb/medleydb/__init__.py:72: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


In [65]:
track = mdb.load_multitracks(['MusicDelta_80sRock'])
t = next(track)

conf_df = pd.read_csv(t.activation_conf_v2_fpath)

columns = conf_df.columns
num_instruments = len(columns)
start_time = [0] * num_instruments
new_bin = [False] * num_instruments
source_activations = [] 
s_i = extract_stem_instruments(t)

for index, row in conf_df.iterrows():
    time = row[0]
    stem = 1
    for conf in row[1:]:
        if not conf >= 0.5 and new_bin[stem]:
            new_bin[stem] = False
            for instrument_label in s_i[columns[stem]]: 
                source_activations.append((start_time[stem], time-0.0464, instrument_label))
        if conf >= 0.5 and not new_bin[stem]:
            new_bin[stem] = True
            start_time[stem] = time
        stem += 1
        
source_df = pd.DataFrame(source_activations, columns=['start_time', 'end_time', 'instrument_label']) 
source_df = source_df.sort_values(by=['instrument_label'])

homedir = os.path.expanduser("~")
# Top level directory will look like "spectrogram-len_n-fft_hop-length"
base_path = os.path.join(homedir, "data/processed/source_ids")

if not(os.path.exists(base_path)):
        os.makedirs(base_path)    

source_df.to_csv(path_or_buf = os.path.join(base_path, "%s_SOURCEID.lab" % t.track_id), index=False, float_format='%.4f')

In [70]:
source_df.head(10)
print(t.activation_conf_fpath)

/home/carlnys/.local/lib/python3.7/site-packages/medleydb/data/Annotations/Activation_Confidence/original/MusicDelta_80sRock_ACTIVATION_CONF.lab


FileNotFoundError: [Errno 2] File b'poop' does not exist: b'poop'

In [4]:
track = mdb.load_multitracks(['MusicDelta_80sRock'])
t = next(track)

def create_source_from_activations(t, stems=None):
    
    try: 
        conf_df = pd.read_csv(t.activation_conf_v2_fpath)
    except FileNotFoundError: 
        conf_df = pd.read_csv(t.activation_conf_fpath)
    columns = conf_df.columns
    num_instruments = len(columns)
    start_time = [0] * num_instruments
    new_bin = [False] * num_instruments
    source_activations = [] 
    s_i = extract_stem_instruments(t)
    
    for index, row in conf_df.iterrows():
        time = row[0]
        stem = 1
        for conf in row[1:]:
            if not conf >= 0.5 and new_bin[stem]:
                new_bin[stem] = False
                for instrument_label in s_i[columns[stem]]: 
                    source_activations.append((start_time[stem], time-0.0464, instrument_label))
            if conf >= 0.5 and not new_bin[stem]:
                new_bin[stem] = True
                start_time[stem] = time
            stem += 1
            
    source_df = pd.DataFrame(source_activations, columns=['start_time', 'end_time', 'instrument_label']) 
    source_df = source_df.sort_values(by=['instrument_label'])
    
    homedir = os.path.expanduser("~")
    # Top level directory will look like "spectrogram-len_n-fft_hop-length"
    base_path = os.path.join(homedir, "data/processed/source_ids")
    
    if not(os.path.exists(base_path)):
            os.makedirs(base_path)    
    
    source_df.to_csv(path_or_buf = os.path.join(base_path, "%s_SOURCEID.lab" % t.track_id), index=False, float_format='%.4f')

In [5]:
def preprocess_all_data(datasets = ['V1', 'V2']):
    track_generator = mdb.load_all_multitracks(datasets)
    count = 1
    for t in track_generator:
        print("Track_ID: %s Count: %d" % (t.track_id.ljust(50, " "), count), end="\r")
        if os.path.isfile(t.activation_conf_v2_fpath) or os.path.isfile(t.activation_conf_fpath):
            create_source_from_activations(t)
            count += 1
        else:
            print ("File does not exist: ", t.activation_conf_v2_fpath)

In [132]:
preprocess_all_data()

File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_Confidence/v2/Allegria_MendelssohnMovement1_ACTIVATION_CONF.lab
File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_Confidence/v2/AmadeusRedux_MozartAllegro_ACTIVATION_CONF.lab
File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_Confidence/v2/AmadeusRedux_SchubertMovement2_ACTIVATION_CONF.lab
File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_Confidence/v2/AmadeusRedux_SchubertMovement3_ACTIVATION_CONF.lab
File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_Confidence/v2/DahkaBand_SoldierMan_ACTIVATION_CONF.lab
File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_Confidence/v2/DeclareAString_MendelssohnPianoTrio1Movement1_ACTIVATION_CONF.lab
File does not exist:  /home/carlnys/data/medleydb/medleydb/data/Annotations/Activation_C

In [16]:
def compare_source_files(t, precision=1):
    file_name = "%s_SOURCEID.lab" % t.track_id
    homedir = os.path.expanduser("~")
    file_path = os.path.join(homedir, "data/processed/source_ids", file_name)
    custom = pd.read_csv(file_path, index_col=False)
    custom = custom.sort_values(by=['instrument_label', 'end_time']).round(precision).reset_index(drop=True)
    #custom = custom.sum(axis = 0, numeric_only = True)
    #print(custom.head(10))
    medley = pd.read_csv(t.source_id_fpath, index_col=False)
    medley = medley.sort_values(by=['instrument_label', 'end_time']).round(precision).reset_index(drop=True)
    #medley = medley.sum(axis = 0, numeric_only = True)
    #sub = custom.sub(medley).sum(axis=0)
    
    print(custom.head(10), "\n")
    print(medley.head(10), "\n")
    print(custom.equals(medley))
    #print(sub)
    #print(sub.sum(axis=0), "\n==============================================")
    #print(abs(sub))
    #if abs(sub) < 1:
    #    return 1
    #else:
    #    #print(t.source_id_fpath)
    #    return 0 
    return 1

In [17]:
track_generator = mdb.load_all_multitracks(['V1'])
count = 1
sum = 0

for t in track_generator:
    sum += compare_source_files(t, 4)
    count += 1
    
print("%d/%d songs sourced the same" % (sum, count))

   start_time  end_time           instrument_label
0      0.9752  139.8306      clean electric guitar
1      0.0464    4.8762  distorted electric guitar
2     18.7617   23.1736  distorted electric guitar
3     37.9414   42.4461  distorted electric guitar
4     96.1771  135.0473  distorted electric guitar
5     94.6445  135.9297  distorted electric guitar
6      0.0000  168.7162  distorted electric guitar
7    133.1897  171.3169  distorted electric guitar
8      0.0929   57.9570                   drum set
9     58.3750  168.2518                   drum set 

   start_time  end_time           instrument_label
0      0.9752  139.8306      clean electric guitar
1      0.0464    4.8762  distorted electric guitar
2     18.7617   23.1735  distorted electric guitar
3     37.9414   42.3996  distorted electric guitar
4     96.1771  135.0473  distorted electric guitar
5     94.6445  135.9296  distorted electric guitar
6      0.0000  168.7162  distorted electric guitar
7    133.1897  171.4097  dist

   start_time  end_time       instrument_label
0      2.0434  246.0387  clean electric guitar
1      2.1362  245.8994           drum machine
2      0.1393  248.2214               mandolin
3     13.0032  245.6672                  tabla 

   start_time  end_time       instrument_label
0      2.0434  246.0386  clean electric guitar
1      2.1362  245.8993           drum machine
2      0.1393  248.2213               mandolin
3     13.0032  245.6671                  tabla 

False
   start_time  end_time instrument_label
0     21.7803  361.7669    electric bass
1      0.6037  168.1590        kick drum
2    208.6545  330.2807        kick drum
3      0.2322  350.7142         mandolin
4     22.4305  360.4666            tabla 

   start_time  end_time instrument_label
0     21.7803  361.7669    electric bass
1      0.6037  168.1589        kick drum
2    208.6545  330.2806        kick drum
3      0.2322  350.6678         mandolin
4     22.4305  360.4666            tabla 

False
   start_time  end

   start_time  end_time instrument_label
0      0.0000   32.3687     drum machine
1     33.9476   67.7559     drum machine
2     68.2202  145.8214     drum machine
3    146.5644  200.8991     drum machine
4      4.2725    6.5945    electric bass
5      6.8731   19.1333    electric bass
6     20.8515   28.0962    electric bass
7     32.0900   35.9910    electric bass
8     36.1302   46.8579    electric bass
9     53.2666   56.0066    electric bass 

   start_time  end_time instrument_label
0      0.0000   32.3686     drum machine
1     33.9476   67.7558     drum machine
2     68.2202  145.8213     drum machine
3    146.5644  200.8990     drum machine
4      4.2725    6.5945    electric bass
5      6.8731   19.1332    electric bass
6     20.8515   28.0961    electric bass
7     32.0900   35.9909    electric bass
8     36.1302   46.8579    electric bass
9     53.2666   56.0065    electric bass 

False
   start_time  end_time instrument_label
0     23.2200   43.7929    female singer
1     

   start_time  end_time instrument_label
0     27.6782   44.0251          bassoon
1     44.6752   54.2419          bassoon
2     82.1058   91.3473          bassoon
3     92.7869  100.6818          bassoon
4    102.3536  120.5116          bassoon
5    214.3202  225.2800          bassoon
6    319.7852  325.1723          bassoon
7    332.1847  342.5873          bassoon
8    346.8597  377.9745          bassoon
9    398.3151  406.4886          bassoon 

   start_time  end_time instrument_label
0     27.6782   44.0250          bassoon
1     44.6752   54.2418          bassoon
2     82.1058   91.3473          bassoon
3     92.7869  100.6817          bassoon
4    102.3536  120.5116          bassoon
5    214.3202  225.2800          bassoon
6    319.7852  325.1722          bassoon
7    332.1847  342.5872          bassoon
8    346.8597  377.9744          bassoon
9    398.3151  406.4885          bassoon 

False
   start_time  end_time instrument_label
0     73.0964   74.9541    brass section
1     

   start_time  end_time instrument_label
0     18.6224   20.5729             dizi
1     21.4088   24.1488             dizi
2     32.6008   40.9136             dizi
3     42.0281   43.0498             dizi
4     44.2572   45.2790             dizi
5     46.4399   47.4616             dizi
6     48.5761   49.5979             dizi
7     50.2944   52.6629             dizi
8     53.0344   61.0685             dizi
9      0.0929   60.3719             erhu 

   start_time  end_time instrument_label
0     18.6224   20.5729             dizi
1     21.4088   24.1488             dizi
2     32.6008   40.9136             dizi
3     42.0281   43.0498             dizi
4     44.2572   45.2789             dizi
5     46.4399   47.4616             dizi
6     48.5761   49.5978             dizi
7     50.2944   52.6629             dizi
8     53.0344   61.0685             dizi
9      0.0929   60.3719             erhu 

False
   start_time  end_time instrument_label
0      0.1858    3.1115             dizi
1     

   start_time  end_time instrument_label
0      9.1487   14.1642   alto saxophone
1     17.8329   24.8454   alto saxophone
2     25.9599   33.3439   alto saxophone
3     34.4584   49.2263   alto saxophone
4      0.0000   49.0406         drum set
5      0.0464   49.1799    electric bass
6      0.0000   49.3192   electric piano 

   start_time  end_time instrument_label
0      9.1487   14.1642   alto saxophone
1     17.8329   24.8454   alto saxophone
2     25.9599   33.3439   alto saxophone
3     34.4584   49.3192   alto saxophone
4      0.0000   49.0870         drum set
5      0.0464   49.2263    electric bass
6      0.0000   49.3656   electric piano 

False
   start_time  end_time instrument_label
0      0.0464  106.3010         drum set
1      0.0000   33.7154    electric bass
2     34.3655  108.1586    electric bass
3      0.0000    6.7338   electric piano
4      7.2911   23.1736   electric piano
5     24.7060   26.4243   electric piano
6     27.0745   31.7185   electric piano
7     

Empty DataFrame
Columns: [start_time, end_time, instrument_label]
Index: [] 

   start_time  end_time instrument_label
0      0.2322   180.419  acoustic guitar
1      2.1827   180.419            flute
2      2.0898   180.419           violin 

False
   start_time  end_time instrument_label
0      0.1858  189.6142  acoustic guitar
1    116.6106  189.1498    electric bass
2      8.9629   20.8980            flute
3     20.9908   83.4526            flute
4     84.1027   86.3783            flute
5     86.7033   94.8768            flute
6     95.2018   97.5703            flute
7     98.7777  100.4960            flute
8    103.5610  157.8493            flute
9    157.9886  167.3230            flute 

   start_time  end_time instrument_label
0      0.1858  189.6141  acoustic guitar
1    116.6106  189.1498    electric bass
2      8.9629   20.8980            flute
3     20.9908   83.4525            flute
4     84.1027   86.3782            flute
5     86.7033   94.8767            flute
6     95.2

   start_time  end_time    instrument_label
0      0.0000  208.2366            drum set
1    223.7475  320.2032            drum set
2      2.6935   16.1611  fx/processed sound
3     34.7371   48.2047  fx/processed sound
4     64.5979   66.2234  fx/processed sound
5     66.6413   68.2203  fx/processed sound
6     68.5917   70.1707  fx/processed sound
7     70.6351   72.1677  fx/processed sound
8     72.7713   74.0717  fx/processed sound
9     74.7218   76.0686  fx/processed sound 

   start_time  end_time    instrument_label
0      0.0000  208.2366            drum set
1    223.7475  320.2032            drum set
2      2.6935   16.1611  fx/processed sound
3     34.7371   48.2046  fx/processed sound
4     64.5979   66.2233  fx/processed sound
5     66.6413   68.2202  fx/processed sound
6     68.5917   70.1707  fx/processed sound
7     70.6351   72.1676  fx/processed sound
8     72.7713   74.0717  fx/processed sound
9     74.7218   76.0686  fx/processed sound 

False
   start_time  end_tim

In [99]:
compare_source_files(t,4)

start_time    0.0000
end_time      0.0003
dtype: float64


0